# TODO 
use xxx model from [TensorFlow detection zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)
And analyse it.

### Prerequirements
Prerequirement for this is TensorFlow 2.X and object_detection API installed
```bash
git clone https://github.com/tensorflow/models.git tf_models
cd tf_models/research
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
sudo pip3 install .
````


In [2]:
import os
import tensorflow as tf
import numpy as np

from jetutils import SimpleTimer

First download and convert a TensorFlow object detection pretrained model. This takes significant amount of time

Like: `
model converted in 823.534474207 seconds`


In [ ]:
MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'
MODEL_NAME = MODEL_URL.split('/')[-1].split('.')[0]
MODEL_PATH = './models/{}'.format(MODEL_NAME)
TRT_MODEL_PATH ='./models/{}_trt'.format(MODEL_NAME)


if not os.path.exists(MODEL_PATH):
    import subprocess
    subprocess.call(['wget', MODEL_URL, '-O', '{}.tgz'.format(MODEL_PATH)])
    subprocess.call(['tar', '-xzf', '{}.tgz'.format(MODEL_PATH), '-C', './models'])

if not os.path.exists(TRT_MODEL_PATH):
    from tensorflow.python.compiler.tensorrt import trt_convert as trt
    from jetutils import CAPTURE_WIDTH, CAPTURE_HEIGHT
    BATCH_SIZE = 1
    ITERATIONS = 10
    for gpu in tf.config.experimental.list_physical_devices('GPU'):
        print(gpu)
        tf.config.experimental.set_memory_growth(gpu, True)

    conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
    conversion_params = conversion_params._replace(precision_mode=trt.TrtPrecisionMode.FP16)
    conversion_params = conversion_params._replace(minimum_segment_size=50)
    conversion_params = conversion_params._replace(max_batch_size=BATCH_SIZE)
  
    def get_dataset(batch_size, input_h, input_w):
        features = np.random.normal(loc=112, scale=70,
              size=(batch_size, input_h, input_w, 3)).astype(np.float32)
        features = np.clip(features, 0.0, 255.0).astype(np.uint8)
        features = tf.convert_to_tensor(value=tf.compat.v1.get_variable(
            "features", initializer=tf.constant(features)))
        dataset = tf.data.Dataset.from_tensor_slices([features])
        dataset = dataset.repeat()
        return dataset

    def input_fn():
        dataset = get_dataset(BATCH_SIZE, CAPTURE_HEIGHT, CAPTURE_WIDTH)
        for i, batch in enumerate(dataset):
            if i > ITERATIONS:
                break
            yield (batch,)
            print(' step {}'.format(i+1))

    converter = trt.TrtGraphConverterV2(
        input_saved_model_dir='{}/saved_model'.format(MODEL_PATH),
        conversion_params=conversion_params)

    timer = SimpleTimer()
    print('converting...')
    with timer:
        converter.convert()
    print('model converted in {} seconds'.format(timer.time))
    print('building...')
    with timer:
        converter.build(input_fn=input_fn)
    print('model engine build in {} seconds'.format(timer.time))
    converter.save(TRT_MODEL_PATH)
    